In [2]:
#https://learn.microsoft.com/en-us/semantic-kernel/concepts/kernel?pivots=programming-language-python

In [31]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.mistral_ai import MistralAIChatCompletion
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from sentence_transformers import SentenceTransformer
import chromadb
import asyncio  # Import asyncio

# Initialize the Kernel
kernel = Kernel()

# Set up the Mistral AI service for chat completion
chat_completion = MistralAIChatCompletion(
    ai_model_id="mistral-small-latest",
    api_key="Md5ifgn7nuQjtrOOsx69vlP00X3g2lzr",
    service_id="chatbot"
)

# Register the service
kernel.add_service(chat_completion)

# Initialize SentenceTransformer model for text embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to get text embeddings
def get_text_embeddings(sentences):
    return model.encode(sentences)

# Initialize ChromaDB client
db_client = chromadb.PersistentClient(path="./chroma_db")
collection = db_client.get_or_create_collection(name="rag_documents")

# Function to add documents to the vector DB
def add_to_vector_db(texts, ids):
    embeddings = get_text_embeddings(texts)
    collection.add(
        ids=[str(i) for i in ids],
        embeddings=embeddings,
        metadatas=[{"text": text} for text in texts]
    )

# Function to retrieve similar texts from the vector DB
def retrieve_similar_texts(query, top_k=1):  # Set top_k to 1 for single retrieval
    query_embedding = get_text_embeddings([query])[0]
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        include=["metadatas"]
    )
    if results and results["metadatas"]:
        return [metadata["text"] for metadata in results["metadatas"][0]]
    else:
        return []

# Define prompt settings
prompt_setting = kernel.get_prompt_execution_settings_from_service_id("chatbot")
prompt_setting.max_tokens = 100
prompt_setting.temperature = 0.5

# Sample texts and IDs
sample_texts = [
    "Renewable energy is derived from natural processes that are replenished constantly.",
    "Solar energy is a type of renewable energy that harnesses the power of the sun.",
    "Wind energy uses wind turbines to convert kinetic energy into electricity."
]
sample_ids = [1, 2, 3]
add_to_vector_db(sample_texts, sample_ids)

# User input query
user_input = "Tell me about renewable energy."

# Retrieve similar text to the user query
retrieved_texts = retrieve_similar_texts(user_input, top_k=1)

# Check if any text is retrieved and pass it to the LLM for an answer
if retrieved_texts:
    prompt_text = f"Given the following information, please answer the question: {user_input}\n\n{retrieved_texts[0]}"
    
    # Define the prompt template with retrieved text
    prompt_template = PromptTemplateConfig(
        template=prompt_text,
        name="Test1",
        template_format="semantic-kernel",
        execution_settings=prompt_setting
    )

    # Register the prompt function
    prompt_function = kernel.add_function(
        function_name="prompt_function",
        plugin_name="semantic_kernel_plugin",
        prompt_template_config=prompt_template
    )

    # Use await directly to invoke the async function
    async def get_llm_answer():
        result = await kernel.invoke(prompt_function)
        print(result)

    # Run the async function within the notebook environment
    await get_llm_answer()

else:
    print("No similar text found.")


Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3


Renewable energy refers to energy sources that are naturally replenished on a human timescale. These sources are virtually inexhaustible, unlike fossil fuels like coal, oil, and natural gas. Here are some key aspects of renewable energy:

1. **Types of Renewable Energy**:
   - **Solar Energy**: Harnessed from the sun's radiation using technologies like solar panels (photovoltaic cells) or concentrated solar


## Flow

# Setup of KB

In [32]:
from sentence_transformers import SentenceTransformer
import chromadb

# Initialize SentenceTransformer model for text embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Initialize ChromaDB client (persistent storage)
db_client = chromadb.PersistentClient(path="./chroma_db")  # Specify the path to save the ChromaDB
collection = db_client.get_or_create_collection(name="rag_documents")

# Function to get text embeddings using SentenceTransformer
def get_text_embeddings(sentences):
    return model.encode(sentences)

# Function to add texts and their embeddings to ChromaDB
def add_to_vector_db(texts, ids):
    embeddings = get_text_embeddings(texts)  # Create embeddings for texts
    collection.add(
        ids=[str(i) for i in ids],  # IDs must be strings
        embeddings=embeddings,
        metadatas=[{"text": text} for text in texts]  # Attach original text as metadata
    )

# Sample knowledge base data (expand as needed)
sample_texts = [
    "Renewable energy is derived from natural processes that are replenished constantly.",
    "Solar energy is a type of renewable energy that harnesses the power of the sun.",
    "Wind energy uses wind turbines to convert kinetic energy into electricity.",
    "Geothermal energy utilizes heat from beneath the earth's surface.",
    "Hydropower generates electricity from the energy of flowing water.",
    "Biomass energy comes from organic materials such as plants and waste.",
    "Nuclear energy is produced by splitting atoms in a controlled environment.",
    "Electric cars are powered by electricity instead of gasoline.",
    "Energy storage technologies help store excess energy for later use.",
    "Sustainability focuses on meeting the needs of the present without compromising future generations."
]

sample_ids = [i for i in range(1, len(sample_texts) + 1)]  # Generate unique IDs for each sample text

# Add sample data to the vector DB
add_to_vector_db(sample_texts, sample_ids)

print("Data has been successfully added to the ChromaDB vector database.")

Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3


Data has been successfully added to the ChromaDB vector database.


# Q&A

In [67]:
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.mistral_ai import MistralAIChatCompletion
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from sentence_transformers import SentenceTransformer
import chromadb
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import ChatMessageContent, TextContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.core_plugins import TimePlugin 

# Initialize the Kernel
kernel = Kernel()

# Set up the AI model (Mistral AI)
chat_completion = MistralAIChatCompletion(
    ai_model_id="mistral-small-latest",
    api_key="Md5ifgn7nuQjtrOOsx69vlP00X3g2lzr",
    service_id="chatbot"
)

# Register the service
kernel.add_service(chat_completion)

# Initialize SentenceTransformer model for text embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def get_text_embeddings(sentences):
    return model.encode(sentences)

# Initialize ChromaDB client
db_client = chromadb.PersistentClient(path="./chroma_db")
collection = db_client.get_or_create_collection(name="rag_documents")

def retrieve_similar_texts(query, top_k=1):
    query_embedding = get_text_embeddings([query])[0]
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        include=["metadatas"]
    )
    if results and results["metadatas"]:
        return [metadata["text"] for metadata in results["metadatas"][0]]
    else:
        return []


# Import Time Plugin
time_plugin = TimePlugin()

# Define prompt settings
prompt_setting = kernel.get_prompt_execution_settings_from_service_id("chatbot")
prompt_setting.max_tokens = 100
prompt_setting.temperature = 0.5

# Initialize Chat History
chat_history = ChatHistory()

async def process_user_query(user_input):
    print("\n🚀 Processing User Query...\n")
    
    # Retrieve similar text from the vector database
    retrieved_texts = retrieve_similar_texts(user_input)
    
    if not retrieved_texts:
        print("⚠️ No relevant text found in the database.")
        return
    
    retrieved_text = "\n".join(retrieved_texts)
    print(f"🔍 Retrieved Text:\n{retrieved_text}\n")
    
    # Append user query to chat history
    chat_history.add_message(
        ChatMessageContent(
            role=AuthorRole.USER,
            items=[TextContent(text=user_input)]
        )
    )

    # Define the prompt template
    prompt_text = f"""Chat History:
    {chat_history}
    
    Answer the following question based on the provided text within 30 words: 
    {retrieved_text}
    
    Question: {user_input}
    
    Answer:"""
    
    # Create prompt template configuration
    prompt_template = PromptTemplateConfig(
        template=prompt_text,
        name="Test1",
        template_format="semantic-kernel",
        execution_settings=prompt_setting
    )
    
    # Register the prompt function
    prompt_function = kernel.add_function(
        function_name="prompt_function",
        plugin_name="semantic_kernel_plugin",
        prompt_template_config=prompt_template
    )
    
    # Wrap the arguments in KernelArguments
    arguments = KernelArguments(
        retrieved_text=retrieved_text,
        user_query=user_input
    )
    
    print("📌 Arguments: ", arguments)
    print("📌 Prompt Function: ", prompt_function)
    
    # Invoke the model
    result = await kernel.invoke(prompt_function, arguments=arguments)
    
    # Ensure the response is a string
    ai_response = str(result) if not isinstance(result, str) else result

    # Append model response to chat history
    chat_history.add_message(
        ChatMessageContent(
            role=AuthorRole.ASSISTANT,
            items=[TextContent(text=ai_response)]
        )
    )
    
    print(f"\n🤖 AI Response:\n{ai_response}\n")

# Run the loop to process user input until 'exit' is typed
async def run_loop():
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Exiting the chat. Goodbye!")
            break
        await process_user_query(user_input)

# Start the loop
asyncio.run(run_loop())

You:  Hi



🚀 Processing User Query...

🔍 Retrieved Text:
Sustainability focuses on meeting the needs of the present without compromising future generations.

📌 Arguments:  {'retrieved_text': 'Sustainability focuses on meeting the needs of the present without compromising future generations.', 'user_query': 'Hi'}
📌 Prompt Function:  metadata=KernelFunctionMetadata(name='prompt_function', plugin_name='semantic_kernel_plugin', description=None, parameters=[], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None, schema_data=None, include_in_function_choices=True), additional_properties=None) invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x0000027B91F22910> streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x0000027B8BCB7FD0> prompt_template=Ke

You:  Tell me about renewable energy sources.



🚀 Processing User Query...

🔍 Retrieved Text:
Solar energy is a type of renewable energy that harnesses the power of the sun.

📌 Arguments:  {'retrieved_text': 'Solar energy is a type of renewable energy that harnesses the power of the sun.', 'user_query': 'Tell me about renewable energy sources.'}
📌 Prompt Function:  metadata=KernelFunctionMetadata(name='prompt_function', plugin_name='semantic_kernel_plugin', description=None, parameters=[], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None, schema_data=None, include_in_function_choices=True), additional_properties=None) invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x0000027B8BF68750> streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x0000027B868F76D0> prompt_template=Kerne

You:  exit


Exiting the chat. Goodbye!


In [46]:
#https://github.com/inkri/Semantic-Kernel/blob/main/SK_Quick_Start.ipynb

In [47]:
#https://github.com/john0isaac/rag-semantic-kernel-mongodb-vcore/blob/main/rag-azure-openai-cosmosdb-notebook.ipynb

In [56]:
#https://github.com/Azure-Samples/semantic-kernel-rag-chat